## Big Data Group I : Intelligent Surveillance -- Image Tagging Process

### Import the necessary packages

In [4]:
import face_recognition
import argparse
import pickle
import cv2
import os
import errno
import dlib
from PIL import Image
import numpy as np

import pandas as pd

import cv2
import random
import shutil

import openface
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.python.keras.models import load_model
from keras.models import load_model
print(tf.__version__)

/home/rajesh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.5.0


Using TensorFlow backend.


#### This function is used to create the folder if it doesn't exist
#### If the directory already exists, don't do anything.

In [5]:
def mkdirP(path):
    """
    Create a directory and don't error if the path already exists.

    If the directory already exists, don't do anything.

    :param path: The directory to create.
    :type path: str
    """
    assert path is not None

    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

#### Get all the Names from the csv file and make a dictionary of Name and the label number

In [6]:
def getNames(filePath):
    personNameFile = pd.read_csv(filePath)
    dict_labels={}
    for (name, category) in zip(personNameFile['Name'] ,personNameFile['personCategory']):
        dict_labels[category]=name
    return dict_labels

#### Load the keras model which was built in the BuildCNNImgClsModel into model variable

In [7]:
def getModel():
    model = load_model('./tagFaceFinalModel.keras')
    return model

#### Read the image from the path and put it into image variable after 
#### resizing to 480,480 pixels and convert the image into rgb format

In [8]:
def readImage(imagePath):
    image = cv2.imread(imagePath)
    image = cv2.resize(image,(480,480))
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image, rgb

#### Get the indices of the boxes for the recognized faces using face_recognition package

In [9]:
def getFaceBoxes(rgbImage):
    boxes = face_recognition.face_locations(rgbImage,
                                        model="cnn")
    return boxes

#### Align the faces that were identified in the original image

In [10]:
def getAlignedImage(faceImage, size):
    outRgb = align.align(size, faceImage,landmarkIndices=landmarkIndices)
    outBgr = None
    if outRgb is not None:
                    outBgr = cv2.cvtColor(outRgb, cv2.COLOR_RGB2BGR)
                    #print(outBgr)
    return outBgr

#### Tag the faces with the names identified by the model. Draw the rectangle around the face and put the name

In [11]:
def tagTheFaces(boxIndices, sourceImage):
    names = []
    for ((top, right, bottom, left)) in boxIndices:
        # draw the predicted face name on the image
        #cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        face_frames = [left, top, right, bottom]
        face = Image.fromarray(sourceImage).crop(face_frames)
        npFace = np.array(face)
        alignedPic = getAlignedImage(npFace,96)
        if alignedPic is not None:
            gray = cv2.cvtColor(alignedPic, cv2.COLOR_BGR2GRAY)/255.0
            train_data = np.array([gray.reshape(96, 96, 1)])
            #print(train_data.shape)
            maxProb = np.max(model.predict(train_data), axis=1)
            #print(maxProb)
            if(maxProb >=0.4):
                name = dict_labels[int(np.argmax(model.predict(train_data), axis=1))]
            else:
                name = "Unknown"
        else:
            name = "UnIdentified"
        #print(name)
        names.append(name)
    return names

#### Save the Tagged image to TagImageDir

In [12]:
def saveTaggedImage(boxIndices, names, sourceImage, savePath):
    # loop over the recognized faces
    #print(boxIndices)
    for ((top, right, bottom, left), name) in zip(boxIndices, names):
        # draw the predicted face name on the image
        cv2.rectangle(sourceImage, (left, top), (right, bottom), (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(sourceImage, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
                    0.75, (0, 255, 0), 2)

    # show the output image
    cv2.imwrite(savePath,sourceImage)

#### Process the Image that were at a specific path and save the image at the path specified

In [13]:
def processImages(imagePath, savePath):
    image,rgb = readImage(imagePath)
    boxes = getFaceBoxes(rgb)
    names = tagTheFaces(boxes, image)
    saveTaggedImage(boxes, names, image, savePath)
    

#### Define the required parameters for aligning the faces.

In [14]:
landmarkMap = {
        'outerEyesAndNose': openface.AlignDlib.OUTER_EYES_AND_NOSE,
        'innerEyesAndBottomLip': openface.AlignDlib.INNER_EYES_AND_BOTTOM_LIP
    }

landmarkIndices = landmarkMap["outerEyesAndNose"]

align = openface.AlignDlib("./openface/models/dlib/shape_predictor_68_face_landmarks.dat")

model = getModel()


#### Read the Names and their label to dictionary object

In [15]:
dict_labels = getNames("./LFWGrayImagesWithPixels_Modified_gt10.csv")

#### Define the input directory of images to be processed and Output Directory for the Tagged Images

In [16]:
dataSetPath = "./imageSet1/"
saveImageDataDir = "./TagImageDir"
# Get the path of all the images from the input Image Directory path
imageName = os.listdir(dataSetPath)
# Create the TagImageDir
mkdirP(saveImageDataDir)

#### Iterate through all the images, process, using the model identify the name, tag and save in output directory

In [17]:
for image in imageName:
            imageFileName  = os.path.join(dataSetPath,image)
            print(imageFileName)
            saveFileName = os.path.join(saveImageDataDir, image)
            #print(saveFileName)
            processImages(imageFileName, saveFileName)

./imageSet1/IMG_4959.JPG
./imageSet1/IMG_4960.JPG
./imageSet1/IMG_4961.JPG
./imageSet1/IMG_4962.JPG
